In [3]:
# Install packages
%pip install networkx
%pip install numpy
%pip install tqdm
%pip install matplotlib

In [4]:
# The starter code is short and simple, take a look!
from starter import *
import random

# Phase 1 - Create Inputs

In [5]:
# Add edges with G.add_edge(u, v, weight=w)
# Idea: consider making the inputs with a solution in mind, 
# such that you know the solution when the outputs are released

def small(G: nx.Graph):
    # TODO add edges to complete the small input
    while G.size(weight = "weight") < 500_000:
        u = random.choice(range(0, N_SMALL))
        v = random.choice(list(range(0, u)) + list(range(u + 1, N_SMALL)))
        w = random.choice(range(1, 1_001))
        G.add_edge(u, v, weight=w)
    
def medium(G: nx.Graph):
    # TODO add edges to complete the medium input
    while G.size(weight = "weight") < 500_000:
        u = random.choice(range(0, N_MEDIUM))
        v = random.choice(list(range(0, u)) + list(range(u + 1, N_MEDIUM)))
        w = random.choice(range(1, 1_001))
        G.add_edge(u, v, weight=w)
    
def large(G: nx.Graph):
    # TODO add edges to complete the large input
    while G.size(weight = "weight") < 500_000:
        u = random.choice(range(0, N_LARGE))
        v = random.choice(list(range(0, u)) + list(range(u + 1, N_LARGE)))
        w = random.choice(range(1, 1_001))
        G.add_edge(u, v, weight=w)

In [6]:
G = nx.empty_graph(N_SMALL)
small(G)
print(validate_input(G))
write_input(G, 'small.in')

G = nx.empty_graph(N_MEDIUM)
medium(G)
print(validate_input(G))
write_input(G, 'medium.in')

G = nx.empty_graph(N_LARGE)
large(G)
print(validate_input(G))
write_input(G, 'large.in')

# Helper Functions #

In [8]:

def make_test():
    test = nx.empty_graph(16)
    for i in range(0, 3):
        test.add_edge(i, i+1, weight=100)
        test.add_edge(i+4, i+4+1, weight=100)
        test.add_edge(i+8, i+8+1, weight=100)
        test.add_edge(i+12, i+12+1, weight=100)
        test.add_edge(i*4, (i+1)*4, weight=100)
        test.add_edge((i*4)+1, ((i+1)*4) + 1, weight=100)
        test.add_edge((i*4)+2, ((i+1)*4) + 2, weight=100)
        test.add_edge((i*4)+3, ((i+1)*4) + 3, weight=100)
    return test

def new_first(G, curr, old_team, new_team, old_first):
    for z in G.neighbors(curr):
        if G.nodes[z]["team"] == old_team:
            old_first -= G[curr][z]["weight"]
        if G.nodes[z]["team"] == new_team:
            old_first += G[curr][z]["weight"]
    return old_first

def new_third(team_sizes, old_team, new_team, old_third, V, k):
    old_b = (math.log(old_third))/B_EXP
    b_i = (team_sizes[old_team]/V) - (1/k)
    b_j = (team_sizes[new_team]/V) - (1/k)
    rad = ((old_b**2) - (b_i**2) - (b_j**2) + (b_i - (1/V))**2 + (b_j + (1/V))**2)
    if rad < 0:
        rad = 0
    return math.exp(B_EXP*math.sqrt(rad))

def DFS_or_BFS(G: nx.Graph, k, sign):
    V = G.number_of_nodes()
    nodes = sorted(G.degree(weight="weight"), key=lambda x: x[1])
    team_sizes = {i:0 for i in range(1, k+1)}
    stack = [i[0] for i in nodes]
    visited = [False]*V
    while stack:
        biggest = stack[-1]
        queue = [biggest]
        while queue:
            curr = queue.pop(sign)
            if not visited[curr]:
                temp = team_sizes.copy()
                for neighbor in G.neighbors(curr):
                    if "team" in G.nodes[neighbor]:
                        adj_team = G.nodes[neighbor]["team"]
                        if adj_team in temp:
                            temp.pop(adj_team)
                if temp:
                    min_team = min(temp, key=temp.get)
                else:
                    edges = G.edges(curr, data=True)
                    adj_team_counter = {i:0 for i in range(1, k+1)}
                    for i in edges:
                        neighbor = i[1]
                        if "team" in G.nodes[neighbor]:
                            adj_team = G.nodes[neighbor]["team"]
                            adj_team_counter[adj_team] += i[2]["weight"]
                    min_team = min(adj_team_counter, key=adj_team_counter.get)
                G.nodes[curr]["team"] = min_team
                team_sizes[min_team] += 1
                stack.remove(curr)
                visited[curr] = True
                for neighbor in list(G.neighbors(curr))[::-1]:
                    if not visited[neighbor]:
                        queue.append(neighbor)
    old_first, old_second, old_third = score(G, separated=True)
    min_score = old_first + old_second + old_third
    for curr in range(V):
        curr_team = G.nodes[curr]["team"] 
        possible_teams = list(range(1, k+1))
        possible_teams.remove(curr_team)
        for neighbor in list(G.neighbors(curr)):
            adj_team = G.nodes[neighbor]["team"]
            if curr_team == adj_team: 
                if adj_team in possible_teams:
                    possible_teams.remove(adj_team)
        for new_team in possible_teams:
            temp_first = new_first(G, curr, curr_team, new_team, old_first)
            temp_third = new_third(team_sizes, curr_team, new_team, old_third, V, k)
            temp_score = temp_first + old_second + temp_third
            if temp_score < min_score:
                old_first = temp_first
                old_third = temp_third
                min_score = temp_score
                curr_team = new_team
        G.nodes[curr]["team"] = curr_team
    return

def partite(G, k):
    best_score = float("inf")
    best_G = G.copy()

    G_copy = G.copy()
    DFS_or_BFS(G_copy, k, 0)
    BFS_score = score(G_copy)
    if BFS_score < best_score:
        best_score = BFS_score
        best_G = G_copy.copy()
    G_copy = G.copy()
    DFS_or_BFS(G_copy, k, -1)
    DFS_score = score(G_copy)
    if DFS_score < best_score:
        best_score = DFS_score
        best_G = G_copy.copy()

    return best_G, best_score


# Phase 2 - Implement your Solver

In [ ]:
''' 
TO RUN ALGORITHM: REPLACE THIS SOLVE FUNCTION WITH THE SOLVE FUNCTION BEING TESTED
'''
def solve(G: nx.Graph):
    V = G.number_of_nodes()
    upper_k = 2*int(((V+1)**(1/2)))
    min_cost = float("inf")
    min_G = G.copy()
    possible_k = range(1, upper_k)
    nodes = sorted(G.degree(weight="weight"), key=lambda x: x[1])
    biggest_node = sorted(G.degree(weight="weight"), key=lambda x: x[1])[-1][0]
    
    for k in possible_k:
        G_copy = G.copy()
        curr_G, curr_score = partite(G_copy, k)
        if curr_score < min_cost:
            min_cost = curr_score
            min_G = curr_G.copy()

    for k in possible_k:
        G_copy = G.copy()
        team_sizes = {i:0 for i in range(1, k+1)}
        '''
        DFS TO PREPROCESS ALL G_COPY NDOES TO MIN_TOT_TEAM
        '''
        visited = [False]*V
        stack = [i[0] for i in nodes]
        while stack:
            biggest_node = stack[-1]
            queue = [biggest_node]
            while queue:
                curr = queue.pop()
                # old_team = G_copy.nodes[curr]["team"]
                if not visited[curr]:
                    edges = sorted(G_copy.edges(curr, data=True), key=lambda t: t[2].get('weight', 1), reverse=True)
                    for i in edges:
                        neighbor = i[1]
                        if not visited[neighbor]:
                            queue.append(neighbor)
                    min_team = min(team_sizes, key=team_sizes.get)
                    G_copy.nodes[curr]["team"] = min_team
                    team_sizes[min_team] += 1
                    # team_sizes[old_team] -= 1
                    stack.remove(curr)
                    visited[curr] = True
        # '''
        # DFS TO PREPROCESS ALL G_COPY NODES TO MIN_ADJ_TEAM 
        # '''
        # visited = [False]*V
        # stack = [i[0] for i in nodes]
        # while stack:
        #     biggest_node = stack[-1]
        #     queue = [biggest_node]
        #     while queue:
        #         curr = queue.pop()
        #         old_team = G_copy.nodes[curr]["team"]
        #         if not visited[curr]:
        #             edges = sorted(G_copy.edges(curr, data=True), key=lambda t: t[2].get('weight', 1))
        #             adj_team_counter = {i:0 for i in range(1, k+1)}
        #             for i in edges:
        #                 neighbor = i[1]
        #                 if "team" in G_copy.nodes[neighbor]:
        #                     adj_team = G_copy.nodes[neighbor]["team"]
        #                     adj_team_counter[adj_team] += i[2]["weight"]
        #                 if not visited[neighbor]:
        #                     queue.append(neighbor)
        #             smallest_team = min(adj_team_counter, key=adj_team_counter.get)
        #             G_copy.nodes[curr]["team"] = smallest_team
        #             team_sizes[smallest_team] += 1
        #             team_sizes[old_team] -= 1
        #             stack.remove(curr)
        #             visited[curr] = True     
        '''
        ITERATE THROUGH SORTED NODES AND CHOOSE BETWEEN MIN_ADJ_TEAM AND MIN_TOT_TEAM
        '''
        for _ in range(k):
            curr_first, curr_second, curr_third = score(G_copy, separated=True)
            curr_score = curr_first + curr_second + curr_third
            queue = [i[0] for i in nodes][::-1]
            for curr in queue:
                edges = G_copy.edges(curr, data=True)
                adj_team_counter = {i:0 for i in range(1, k+1)}
                for i in edges:
                    neighbor = i[1]
                    adj_team = G_copy.nodes[neighbor]["team"]
                    adj_team_counter[adj_team] += i[2]["weight"]
                old_team = G_copy.nodes[curr]["team"]
                min_adj_team = min(adj_team_counter, key=adj_team_counter.get)
                min_tot_team = min(team_sizes, key=team_sizes.get)
                
                adj_first = new_first(G_copy, curr, old_team, min_adj_team, curr_first)
                adj_third = new_third(team_sizes, old_team, min_adj_team, curr_third, V, k)
                adj_score = adj_first + curr_second + adj_third

                tot_first = new_first(G_copy, curr, old_team, min_tot_team, curr_first)
                tot_third = new_third(team_sizes, old_team, min_tot_team, curr_third, V, k)
                tot_score = tot_first + curr_second + tot_third

                if adj_score < curr_score:
                    G_copy.nodes[curr]["team"] = min_adj_team
                    team_sizes[min_adj_team] += 1
                    team_sizes[old_team] -= 1
                    curr_first = adj_first
                    curr_third = adj_third
                    curr_score = adj_score 
                old_team = G_copy.nodes[curr]["team"] 
                if tot_score < curr_score:
                    G_copy.nodes[curr]["team"] = min_tot_team
                    team_sizes[min_tot_team] += 1
                    team_sizes[old_team] -= 1
                    curr_first = tot_first
                    curr_third = tot_third
                    curr_score = tot_score
        '''
        DFS AND ASSIGN NODES BY CHOOSING BETWEEN MIN_ADJ_TEAM AND MIN_TOT_TEAM
        '''
        for _ in range(k):
            curr_first, curr_second, curr_third = score(G_copy, separated=True)
            curr_score = curr_first + curr_second + curr_third
            visited = [False]*V
            stack = [i[0] for i in nodes]
            while stack:
                biggest_node = stack[-1]
                queue = [biggest_node]
                while queue:
                    curr = queue.pop()
                    old_team = G_copy.nodes[curr]["team"]
                    if not visited[curr]:
                        edges = sorted(G_copy.edges(curr, data=True), key=lambda t: t[2].get('weight', 1))
                        adj_team_counter = {i:0 for i in range(1, k+1)}
                        for i in edges:
                            neighbor = i[1]
                            adj_team = G_copy.nodes[neighbor]["team"]
                            adj_team_counter[adj_team] += i[2]["weight"]
                            if not visited[neighbor]:
                                queue.append(neighbor)
                        min_adj_team = min(adj_team_counter, key=adj_team_counter.get)
                        min_tot_team = min(team_sizes, key=team_sizes.get)
                        
                        adj_first = new_first(G_copy, curr, old_team, min_adj_team, curr_first)
                        adj_third = new_third(team_sizes, old_team, min_adj_team, curr_third, V, k)
                        adj_score = adj_first + curr_second + adj_third

                        tot_first = new_first(G_copy, curr, old_team, min_tot_team, curr_first)
                        tot_third = new_third(team_sizes, old_team, min_tot_team, curr_third, V, k)
                        tot_score = tot_first + curr_second + tot_third

                        if adj_score < curr_score:
                            G_copy.nodes[curr]["team"] = min_adj_team
                            team_sizes[min_adj_team] += 1
                            team_sizes[old_team] -= 1
                            curr_first = adj_first
                            curr_third = adj_third
                            curr_score = adj_score 
                        old_team = G_copy.nodes[curr]["team"]
                        if tot_score < curr_score:
                            G_copy.nodes[curr]["team"] = min_tot_team
                            team_sizes[min_tot_team] += 1
                            team_sizes[old_team] -= 1
                            curr_first = tot_first
                            curr_third = tot_third
                            curr_score = tot_score
                        stack.remove(curr)
                        visited[curr] = True   
                    
        curr_cost = score(G_copy)
        if curr_cost <= min_cost:
            min_cost = curr_cost
            min_G = G_copy.copy()
        else:
            break
    for i in min_G.nodes:
        G.nodes[i]['team'] = min_G.nodes[i]['team']
    return


In [10]:
run(solve, 'small.in', 'small.out')

In [13]:
run_all(solve, 'inputs', 'outputs')
tar("outputs")

  0%|          | 0/780 [00:00<?, ?it/s]

inputs\large1.in: cost 11147.260453600116
inputs\large10.in: cost 11657.633464702434
inputs\large100.in: cost 449.2277266951576
inputs\large101.in: cost 14306.294714663649
inputs\large102.in: cost 17128.914817589965
inputs\large103.in: cost 2017.2493960703473
inputs\large104.in: cost 18193.672749482794
inputs\large105.in: cost 739.905609893065
inputs\large106.in: cost 738.2277266951577
inputs\large107.in: cost 53044.814293170435
inputs\large108.in: cost 272.8281828459045
inputs\large109.in: cost 7190.856866529837
inputs\large11.in: cost 10002.914817589966
inputs\large110.in: cost 272.8281828459045
inputs\large111.in: cost 24014.153561009512
inputs\large112.in: cost 13602.344539167265
inputs\large113.in: cost 272.8281828459045
inputs\large114.in: cost 9924.814293170431
inputs\large115.in: cost 449.2277266951576
inputs\large116.in: cost 39616.039122061105
inputs\large117.in: cost 739.905609893065
inputs\large118.in: cost 32237.50819222289
inputs\large119.in: cost 27368.560881495607
input